In [4]:
import numpy as np
import pandas as pd
from cvxpy import *
from cvxopt import *
from alphamind.api import *
from alphamind.cython.optimizers import QPOptimizer

# Data Preparing
--------------------------

In [31]:
risk_penlty = 0.5
ref_date = '2018-02-08'

engine = SqlEngine()
universe = Universe('custom', ['ashare_ex'])
codes = engine.fetch_codes(ref_date, universe)

risk_cov, risk_exposure = engine.fetch_risk_model(ref_date, codes)
factor = engine.fetch_factor(ref_date, 'EPS', codes)

total_data = pd.merge(factor, risk_exposure, on='code').dropna()
all_styles = risk_styles + industry_styles + macro_styles

risk_exposure_values = total_data[all_styles].values.astype(float)
special_risk_values = total_data['s_srisk'].values.astype(float)
risk_cov_values = risk_cov[all_styles].values

sec_cov_values_full = risk_exposure_values @ risk_cov_values @ risk_exposure_values.T / 10000  + np.diag(special_risk_values ** 2) / 10000
signal_full = total_data['EPS'].values

In [32]:
n = 200

sec_cov_values = sec_cov_values_full[:n, :n]
signal = signal_full[:n]

# Optimizing Weights
-------------------------------------

In [33]:
%%time
w = Variable(n)

lbound = 0.
ubound = 1. / n * 20

objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
constraints = [w >= lbound,
               w <= ubound,
               sum_entries(w) == 1,]

prob = Problem(objective, constraints)

CPU times: user 360 ms, sys: 321 ms, total: 681 ms
Wall time: 48.3 ms


In [34]:
%%time
prob.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -3.236e-01  -9.496e+01  +2e+03  9e-01  1e-02  1e+00  5e+00    ---    ---    1  1  - |  -  - 
 1  -5.482e+00  -1.908e+01  +1e+03  2e-01  9e-04  4e-01  3e+00  0.6534  2e-01   1  2  2 |  0  0
 2  -2.485e+00  -3.161e+00  +1e+02  1e-02  3e-05  8e-02  3e-01  0.9700  8e-02   2  2  2 |  0  0
 3  -2.433e+00  -2.610e+00  +3e+01  3e-03  8e-06  2e-02  6e-02  0.7649  2e-02   2  2  2 |  0  0
 4  -2.416e+00  -2.454e+00  +5e+00  7e-04  2e-06  4e-03  1e-02  0.8454  6e-02   2  2  2 |  0  0
 5  -2.408e+00  -2.415e+00  +1e+00  1e-04  3e-07  6e-04  2e-03  0.8342  2e-02   2  2  2 |  0  0
 6  -2.403e+00  -2.406e+00  +4e-01  6e-05  1e-07  1e-04  1e-03  0.8410  3e-01   2  3  3 |  0  0
 7  -2.401e+00  -2.401e+00  +5e-02  7e-06  1e-08  1e-05  1e-04  0.8893  3e-03   2  2  2 |  0  0
 8  -2.400e+00  -2.400e+00  +7e-03  1e-06  2e-

-2.4003282365705503

In [35]:
prob.status, prob.value

('optimal', -2.4003282365705503)

In [36]:
%%time
prob.solve(verbose=True, solver='CVXOPT')

     pcost       dcost       gap    pres   dres   k/t
 0: -3.8414e-01 -9.6498e+01  2e+03  1e+01  4e-01  1e+00
 1: -2.2054e-01 -2.8096e+01  2e+02  3e+00  1e-01  2e+00
 2: -7.3888e-01 -2.3177e+01  1e+02  3e+00  1e-01  2e+00
 3: -9.0974e-01 -1.6105e+01  9e+01  2e+00  7e-02  3e+00
 4: -2.5843e+00 -6.4237e+00  1e+01  5e-01  2e-02  1e+00
 5: -2.6397e+00 -2.9412e+00  1e+00  4e-02  2e-03  1e-01
 6: -2.5273e+00 -2.6901e+00  6e-01  2e-02  9e-04  5e-02
 7: -2.4427e+00 -2.5068e+00  2e-01  8e-03  3e-04  2e-02
 8: -2.4035e+00 -2.4090e+00  2e-02  7e-04  3e-05  1e-03
 9: -2.4004e+00 -2.4005e+00  4e-04  1e-05  5e-07  2e-05
10: -2.4003e+00 -2.4003e+00  6e-06  2e-07  9e-09  4e-07
11: -2.4003e+00 -2.4003e+00  1e-07  4e-09  2e-10  7e-09
Optimal solution found.
CPU times: user 1.47 s, sys: 1.61 s, total: 3.08 s
Wall time: 352 ms


-2.4003282579305463

In [37]:
prob.status, prob.value

('optimal', -2.4003282579305463)

In [38]:
%%time
P = matrix(sec_cov_values)
q = -matrix(signal)

G = np.zeros((2*n, n))
h = np.zeros(2*n)
for i in range(n):
    G[i, i] = 1.
    h[i] = 1. / n * 20
    G[i+n, i] = -1.
    h[i+n] = 0.
    
G = matrix(G)
h = matrix(h)
    
A = np.ones((1, n))
b = np.ones(1)

A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0: -4.0275e+01 -8.9373e+01  8e+03  6e+01  5e-16
 1: -2.7029e+00 -8.3780e+01  2e+02  1e+00  1e-15
 2: -1.3699e+00 -2.0914e+01  2e+01  1e-15  2e-15
 3: -1.6193e+00 -6.3167e+00  5e+00  6e-16  2e-15
 4: -1.8992e+00 -4.2870e+00  2e+00  4e-16  1e-15
 5: -2.1306e+00 -3.2594e+00  1e+00  3e-16  8e-16
 6: -2.1625e+00 -2.9783e+00  8e-01  3e-16  6e-16
 7: -2.2529e+00 -2.6835e+00  4e-01  2e-16  6e-16
 8: -2.3100e+00 -2.5413e+00  2e-01  2e-16  4e-16
 9: -2.3407e+00 -2.4723e+00  1e-01  2e-16  4e-16
10: -2.3953e+00 -2.4100e+00  1e-02  3e-16  1e-15
11: -2.4002e+00 -2.4005e+00  2e-04  4e-16  9e-16
12: -2.4003e+00 -2.4003e+00  2e-06  2e-16  8e-16
13: -2.4003e+00 -2.4003e+00  2e-08  2e-16  1e-15
Optimal solution found.
CPU times: user 584 ms, sys: 636 ms, total: 1.22 s
Wall time: 83 ms


In [39]:
%%time
lbound = np.zeros(n)
ubound = np.ones(n) * 20 / n
cons_matrix = np.ones((1, n))
clb = np.ones(1)
cub = np.ones(1)
qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, cons_matrix, clb, cub, 1.)
qpopt.feval()
qpopt.status()

CPU times: user 70 ms, sys: 36.7 ms, total: 107 ms
Wall time: 107 ms


# Performace Timing
-------------------------

In [40]:
import datetime as dt

In [41]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    py_callable(n)
    return (dt.datetime.now() - start).total_seconds()

In [42]:
def cvxpy(n):
    w = Variable(n)

    lbound = 0.
    ubound = 0.01

    objective = Minimize(risk_penlty * quad_form(w, sec_cov_values)  - signal * w)
    constraints = [w >= lbound,
                   w <= ubound,
                   sum_entries(w) == 1,]

    prob = Problem(objective, constraints)
    prob.solve(verbose=False, solver='CVXOPT', display=False)

In [43]:
def cvxopt(n):
    P = matrix(sec_cov_values)
    q = -matrix(signal)

    G = np.zeros((2*n, n))
    h = np.zeros(2*n)
    for i in range(n):
        G[i, i] = 1.
        h[i] = 0.01
        G[i+n, i] = -1.
        h[i+n] = 0.

    G = matrix(G)
    h = matrix(h)

    A = np.ones((1, n))
    b = np.ones(1)

    A = matrix(A)
    b = matrix(b)
    
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

In [44]:
def ipopt(n):
    lbound = np.zeros(n)
    ubound = np.ones(n) * 0.01
    cons_matrix = np.ones((1, n))
    clb = np.ones(1)
    cub = np.ones(1)
    qpopt = QPOptimizer(signal, sec_cov_values, lbound, ubound, cons_matrix, clb, cub, 1.)
    qpopt.feval()

In [45]:
n_steps = list(range(200, 3201, 200))
cvxpy_times = [None] * len(n_steps)
cvxopt_times = [None] * len(n_steps)
ipopt_times = [None] * len(n_steps)
print("{0:<8}{1:>12}{2:>12}{3:>12}".format('Scale(n)', 'cvxpy', 'cvxopt', 'ipopt'))

for i, n in enumerate(n_steps):
    sec_cov_values = sec_cov_values_full[:n, :n]
    signal = signal_full[:n]
    cvxpy_times[i] = time_function(cvxpy, n) * 1000
    cvxopt_times[i] = time_function(cvxopt, n) * 1000
    ipopt_times[i] = time_function(ipopt, n) * 1000
    
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12.2f}".format(n, cvxpy_times[i], cvxopt_times[i], ipopt_times[i]))

Scale(n)       cvxpy      cvxopt       ipopt
200           359.93       28.96       30.79
400          1104.61      436.92       86.73
600          2248.70      544.58       57.86
800          3371.36     1025.66       88.14
1000         6210.00     1657.73      104.53
1200        12045.02     2785.55      132.32
1400        18365.51     4679.09      154.84
1600        24513.72     5827.70      201.17
1800        33475.16    10443.92      309.69
2000        44205.53    12464.22      330.94
2200        59213.54    17986.24      439.36
2400        72602.25    20114.71      562.75
2600        93412.44    24947.24      573.60
2800       111345.23    31156.20      644.45
3000       127577.49    39207.67      804.31
3200       147128.87    47593.30      956.98
